# Ising 1D transverse field model
## Ground state energy, magnetization and susceptibility

### Libraries

In [1]:
## Basic libraries for printing output, plots, etc.
using Printf
using Plots
using Random

## Libraries for Sparse Arrays
using SparseArrays
using ArnoldiMethod # Solves eigenvalue problems with sparse matrices

## Other libraries
using LinearAlgebra
using Statistics # For statistical measurements
using QuadGK # For numerical integration

### Functions for exact ground state

In [2]:
# Redefinition of tensor product
function ⊙(x,y)
    kron(x,y)
end

# Sparse Hamiltonian
function buildHamiltonian_sparse(;N,J,B)
    #  Input: -----------------------------
    #
    #  N: Number of spins in the chain
    #
    #  J: Hamiltonian interaction term
    #
    #  B: Transverse magnetic field
    #  term
    #
    #  Output: ----------------------------
    #
    #  H: Hamiltonian matrix projected
    #  onto spin basis
    
    Id = [[1,0] [0,1]] |> sparse
    σᶻ = [[1,0] [0,-1]] |> sparse
    σˣ = [[0,1] [1,0]] |> sparse
    
    # Coupling term
    first_term = fill(Id,N)
    first_term[1] = σᶻ
    first_term[2] = σᶻ
    
    second_term = fill(Id,N)
    second_term[1] = σˣ
    
    # Initialize sparse zero matrix
    H = spzeros(Float64,2^N,2^N)
    for i in 1:N
        H += -J*foldl(⊙, first_term) -B*foldl(⊙, second_term)
        first_term = circshift(first_term,1)
        second_term = circshift(second_term,1)
    end
    return H
end

# Sparse eigenvalues
function eigen_sparse(x)
    decomp, history = partialschur(x, nev=1, which=:SR); # only solve for the ground state
    vals, vecs = partialeigen(decomp);
    return vals, vecs
end

# Magnetization of a given state
function buildMagnetizations_sparse(state;N)
    # Input: ------------------------------
    #
    # state: column vector or matrix of vectors
    # encoding a projected wavestate onto the
    # spin basis of the Ising model
    #
    # N: Number of spins in the chain
    # 
    # Output: -----------------------------
    #
    # st_mz: value of the longitudinal
    # magnetization
    #
    # st_mx: value of the transversal
    # magnetization
    
    Id = [[1,0] [0,1]] |> sparse
    σᶻ = [[1,0] [0,-1]] |> sparse
    σˣ = [[0,1] [1,0]] |> sparse
    
    mz = fill(Id,N)
    mz[1] = σᶻ
    
    mx = fill(Id,N)
    mx[1] = σˣ
    
    # Build magnetizations
    Mz = spzeros(Float64,2^N,2^N)
    Mx = spzeros(Float64,2^N,2^N)
    for i in 1:N
        Mz += foldl(⊙, mz)/N
        Mx += foldl(⊙, mx)/N
        mz = circshift(mz, 1)
        mx = circshift(mx, 1)
    end
    
    # Calculate magnetization of the state
    st_mz = sqrt(state'*Mz^2*state)
    st_mx = sqrt(state'*Mx^2*state)
    
    return st_mz, st_mx
end

# Magnetization in the thermodynamic limit (integral)
function magnetizationIntegral(;J,B)
    # Input: -----------------------------
    #
    # J: Hamiltonian interaction term
    #
    # B: Transverse magnetic field term
    #
    # Output: ----------------------------
    #
    # in_mz: value of the longitudinal
    # magnetization in the thermodynamic limit
    #
    # in_mx: value of the transversal
    # magnetization in the thermodynamic limit
    
    in_mz = (B < J) ? (1 - B^2/J^2)^(1/8) : 0
    
    in_mx, err = quadgk(k -> (B + J*cos(k))/sqrt(J^2 + B^2 + 2*J*B*cos(k)), 
        0, pi, rtol=1e-10)
    
    return in_mz, in_mx/pi
end;

### RBM Class definition

In [3]:
# RBM wavefunction class
mutable struct RBM
    n_visible::Int
    n_hidden::Int
    params::Vector{ComplexF64}
    
    function RBM(n_visible::Int, n_hidden::Int)
        rbm = new(n_visible, n_hidden, ComplexF64[])
        initialise_RBM(rbm)
        return rbm
    end
end

# Initialize RBM wavefunction
function initialise_RBM(rbm::RBM;σ=0.01)
    # Parameters of the RBM can be complex valued,
    # and are initialized from a random distribution
    b = randn(ComplexF64,rbm.n_visible)
    c = randn(ComplexF64,rbm.n_hidden)
    W = randn(ComplexF64,rbm.n_hidden,rbm.n_visible)
    
    rbm.params = σ*vcat(b,c,reshape(W,:))
end

# Define property-like functions
function b(rbm::RBM)
    # Returns column vector of visible bias
    return rbm.params[1:rbm.n_visible]
end

function c(rbm::RBM)
    # Returns column vector of hidden bias
    nv = rbm.n_visible
    return rbm.params[nv+1 : nv+rbm.n_hidden]
end

function W(rbm::RBM)
    # Returns nh x nv weight matrix
    nv, nh = rbm.n_visible, rbm.n_hidden
    return reshape(rbm.params[nv+nh+1:end], nh, nv)
end
    
# Additional functions that may be useful
function wavefunction(rbm::RBM, s::Vector{Bool})
    # Calculate wavefunction projection onto state s
    return exp(conj(b(rbm))' * s) * prod(cosh(c(rbm) .+ W(rbm) * s)) * 2^rbm.n_hidden
end

function p_ratio(rbm::RBM, s1::Vector{Bool}, s2::Vector{Bool})
    # Ratio of probabilities between states s2 and s1
    f1 = cosh.(c(rbm) .+ W(rbm) * s1)
    f2 = cosh.(c(rbm) .+ W(rbm) * s2)
    log_diff = conj(b(rbm))' * (s2 - s1) + sum(log.(f2 ./ f1))  # log of ratio for numerical stability
    return exp(log_diff)
end

function p_ratios(rbm::RBM, s1::Vector{Bool}, s2::Vector{Vector{Bool}})
    # Ratio of probabilities between list of states s2 and state s1
    return [p_ratio(rbm, s1, s) for s in s2]
end;

### Useful functions for variational monte carlo

In [4]:
# Flip i-th bit of state s
function flip(s::Vector{Bool},i::Int)
    s_flip = copy(s)
    s_flip[i] = 1 - s_flip[i]
    return s_flip
end

# Local energy of the TFI model
function local_energy(s::Vector{Bool},psi::RBM;J,B)
    # Interaction term
    couplings = (s .== circshift(s,1)) .* 2 .- 1
    e_int = -J*sum(couplings)
    
    # Transverse field term
    flipped_states = [flip(s,i) for i in 1:length(s)]
    e_field = -B*sum(p_ratios(psi,s,flipped_states))
    
    return e_int + e_field
end

# Compute the log-derivative of the wavefunction
function variational_derivative(x::Vector{Bool}, psi::RBM)
    theta = c(psi) + W(psi) * x   # Column of length n_hidden
    Ob = x                        # Column of length n_visible
    Oc = tanh.(theta)
    Ow = Oc * x'                  # Matrix of n_hidden x n_visible
    return vcat(Ob, Oc, Ow...)
end

# Compute the covariance between x1 and x2
# Used for calculating the gradient:
#
# F = covariance(conj.(O),EL') -> x1 is num_params x num_samples and EL is 1 x num_samples
#
# and for the Fischer matrix:
#
# S = covariance(conj.(O),O) -> x1, x2 are num_params x num_samples
function covariance(x1, x2)
    samples = size(x1,2)                              # Number of columns (samples) in x1
    m1 = mean(x1, dims=2)                             # This is num_params x 1
    m2 = size(x2,2) > 1 ? mean(x2, dims=2) : mean(x2) # This is either num_params x 1 or a number
    return size(x2,2) > 1 ? (x1 * x2')/samples - m1 * m2' : (x1 * x2)/samples - m1 .* m2
end

# Sample bs states according to our wavefunction
function sample_block(psi::RBM, bs; x0=nothing)
    state = x0 == nothing ? rand(Bool,psi.n_visible) : x0
    states = []
    for _ in 1:bs
        idx = rand(1:psi.n_visible)
        new_state = flip(state, idx)
        if rand() <= abs(p_ratio(psi,state,new_state))^2
            state = copy(new_state)   # Accept new state   
        end
        push!(states,state)
    end
    return states
end;

### Binning analysis functions

In [5]:
# Mean of the time-series x in chunks of given size
function bin_averages(x, bs)
    nb = length(x) ÷ bs
    bin_avg = [mean(x[b*bs+1:(b+1)*bs]) for b in 0:nb-1]
    return hcat(bin_avg...)
end;

## main code

### Ground state and ground energy

Calculate the ground state energy given the number of spins $N$, the coupling interaction term $J$ and the strength of the transverse magnetic field $B$.

In [6]:
# Value of the coupling term and transverse magnetic field
J, B = 2, 1

# Number of spins
N = 2
H = buildHamiltonian_sparse(N=N,J=J,B=B)

ground_energy, ground_state = eigen_sparse(H)

@printf "Ground state energy: %.8f\n" ground_energy[1]
@printf "Ground state:\n"
display(ground_state)

Ground state energy: -4.47213595
Ground state:


4×1 Matrix{Float64}:
 -0.6881909602355869
 -0.16245984811645306
 -0.1624598481164531
 -0.6881909602355866

### Ground state magnetization

Calculate the longitudinal and transversal magnetizations of the ground state.

In [7]:
st_mz, st_mx = buildMagnetizations_sparse(ground_state;N=N)

([0.9732489894677301;;], [0.9999999999999999;;])

### Plot of the phase transition

We calculate the longitudinal magnetization of the ground state for different values of $B/J$ and plot it for many $N$:

In [8]:
# Values of B/J
B_J = 10 .^ range(start=-2., stop=2., length=10)
Ns = 4:4:20

p = plot()
q = plot()
@time for N in Ns
    st_mz = zeros(length(B_J))
    st_mx = zeros(length(B_J))
    for (i,b) in enumerate(B_J)
        H = buildHamiltonian_sparse(N=N,J=1,B=b)
        vals, vecs = eigen_sparse(H)
        ground_state = @view vecs[:,1]
        st_mz[i], st_mx[i] = buildMagnetizations_sparse(ground_state;N=N)
    end
    plot!(p, B_J, st_mz, marker=:circle, label="L = $N", xscale=:log10,
        lw=3, ls=:dash, markersize=6)
    plot!(q, B_J, st_mx, marker=:circle, label="L = $N", xscale=:log10,
        lw=3, ls=:dash, markersize=6)
end

# Finer grid
B_J = 10 .^ range(start=-2., stop=2., length=1000)
in_mz = zeros(length(B_J))
in_mx = zeros(length(B_J))

for (i,b) in enumerate(B_J)
    # Add thermodynamic limit N → ∞
    in_mz[i], in_mx[i] = magnetizationIntegral(J=1,B=b)
end
plot!(p, B_J, in_mz,size=(800,600), c=:black, label="L = ∞", lw=2, 
    xlab="B/J", ylab="m_z", xtickfontsize=14, xguidefontsize=14,
        ytickfontsize=14, yguidefontsize=14, legendfontsize=14,
        fontfamily="helvetica")
plot!(q, B_J, in_mx, size=(800,600), c=:black, label="L = ∞", lw=2,
    xlab="B/J", ylab="m_x", xtickfontsize=14, xguidefontsize=14,
        ytickfontsize=14, yguidefontsize=14, legendfontsize=14,
        fontfamily="helvetica")
savefig(p,"Ground_Longitudinal_Magnetizations.png")
savefig(q,"Ground_Transversal_Magnetizations.png")

113.084772 seconds (2.24 M allocations: 229.257 GiB, 5.41% gc time, 0.60% compilation time: 20% of which was recompilation)


"/home/marcfernandez/Documents/Master-thesis/TFI/Ground_Transversal_Magnetizations.png"

## Variational Monte Carlo using RBM wavefunction

### Monte Carlo sampling by flipping spins randomly

In [ ]:
## Parameters of the problem
n = 2                           # Number of spins
J, B = 2, 1                     # Hamiltonian
n_visible, n_hidden = n, 1      # RBM size: twice as many hidden neurons
psi = RBM(n_visible,n_hidden);  # Randomly initialize our anstaz

### Calculate autocorrelation time of the samples

### Stochastic Reconfiguration (Natural Gradient) learning

In [ ]:
## Learning parameters
learning_iterations = 375
lr = 1e-2
n_blocks = 150
thermalise = Int(0.1*n_blocks)
Nb = n_blocks - thermalise
bs = 5
energies = []

@time for it in 1:learning_iterations
    EL, O = zeros(ComplexF64,Nb), zeros(ComplexF64,length(psi.params),Nb)
    states = sample_block(psi, thermalise*bs)
    state = states[end]
    for k in 1:Nb
        batch = sample_block(psi, bs; x0=state)
        push!(states,batch)
        state = batch[end]
        EL[k] = local_energy(state,psi;J=J,B=B)
        O[:,k] = variational_derivative(state, psi)
    end
    push!(energies,mean(EL))
    
    F = covariance(conj.(O), EL')          # Gradient
    S = covariance(conj.(O), O)            # Fisher info
    Sinv = pinv(S,rtol=1e-5)               # (pseudo)Inversion
    d_params = lr*Sinv * F
    psi.params -= vec(d_params)
end

p3 = plot(real(energies),size=(800,600),grid=true)
plot!(p3,xlab="Learning iteration",xtickfontsize=20,xguidefontsize=20)
plot!(p3,ylab="⟨H⟩",ytickfontsize=20,yguidefontsize=20)

In [ ]:
energy = real(energies[end-100:end])
print("The obtained energy is: $(round(mean(energy),digits=4)) +- $(round(std(energy)/10,digits=4))")